In [2]:
import networkx as nx
import json
from nltk import word_tokenize
from nltk.corpus import stopwords
import sys
import gensim, logging
import re
import os
import treetaggerwrapper
import random

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Slow version of gensim.models.doc2vec is being used


Загружаем топики; создаём словарь вида: обработанный топик - исходный топик

Лемматизатор:

In [2]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')

Обработка включает в себя: токенизацию, удаление знаков препинания и стопслов, лемматизацию, сведение всех слов к нижнему регистру:

In [8]:
def analyze_topic(topic):
    stop = stopwords.words('english')

    tokens = word_tokenize(topic)
    punct = ',.()":;--&?!\'s'
    tokens = [token for token in tokens if token not in punct]
    tokens = [token for token in tokens if token not in stop]
    new_tokens = []
    for token in tokens:
        lemma = tagger.tag_text(token)[0].split('\t')[-1]
        if lemma != '@card@':
            new_tokens.append(lemma)
        else:
            new_tokens.append(token)
    
    return ' '.join(new_tokens).lower()

Загружаем топики из графа:

In [4]:
G = nx.read_gexf('topics.gexf')

In [5]:
topics = G.nodes()

Обрабатываем и записываем в файл:

In [9]:
new_topics = {}
for topic in topics:
    new_topics[analyze_topic(topic)] = topic

In [11]:
new_topics

{'lisbon earthquake portugal 1755': 'Lisbon Earthquake, Portugal, 1755',
 'earthquake': 'Earthquakes',
 'woman': 'Women',
 'ops roman deity': 'Ops (Roman deity)',
 'snake': 'Snakes',
 'eve biblical figure': 'Eve (Biblical figure)',
 'temptation': 'Temptation',
 'weave': 'Weaving',
 'penelope greek mythology': 'Penelope (Greek mythology)',
 'murder': 'Murder',
 'tombs sepulchral monument': 'Tombs & sepulchral monuments',
 'elephant': 'Elephants',
 'book jacket': 'Book jackets',
 'animal fiction': 'Animals--Fiction',
 'book cover': 'Book covers',
 'domestic life': 'Domestic life',
 'dwelling': 'Dwellings',
 'oromo african people': 'Oromo (African people)',
 'canyon': 'Canyons',
 'teenager': 'Teenagers',
 'peddler': 'Peddlers',
 'market': 'Markets',
 'street': 'Streets',
 'art japanese': 'Arts, Japanese',
 'photography': 'Photography',
 'jews': 'Jews',
 'theater': 'Theaters',
 'costume': 'Costume -- ',
 'art modern': 'Art, Modern',
 'art deco': 'Art deco',
 'decoration ornament': 'Decorat

In [12]:
with open('analyzed_topics.json', 'w') as outfile:
    json.dump(new_topics, outfile)

Загружаем словарь вида: обработанный топик - исходный топик:

In [3]:
new_topics = json.loads(open('analyzed_topics.json').read())

Загружаем модель (основанную на английской википедии):

In [4]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format(os.path.join('word2vec_models','enwiki_5_ner.bin'), binary=True)

2017-05-23 16:43:34,816 : INFO : loading projection weights from word2vec_models\enwiki_5_ner.bin
2017-05-23 16:45:13,431 : INFO : loaded (296630, 300) matrix from word2vec_models\enwiki_5_ner.bin


In [6]:
model.init_sims(replace=True)

2017-05-23 16:46:50,161 : INFO : precomputing L2-norms of word weight vectors


Проверяем, есть ли в модели коллокации, разделённые пробелами:

In [4]:
for word in model.vocab:
    if ' ' in word:
        print(word)

Функция ищет topn ближайших по косинусному расстоянию слов; ищет эти слова в топиках; возвращает первые resultsn топиков, в которых нашлись эти "синонимы"

In [45]:
pos_tags = ['ADJ', 'ADV', 'NOUN', 'PROPN', 'VERB']

def sim_word2vec(word, topn=10, resultsn=10):
    global pos_tags
    results = []

    for pos_tag in pos_tags:
#         print(pos_tag)
        word_tagged = word.replace(' ', '::') + '_' + pos_tag
        if word_tagged in model:
            for i in model.most_similar(positive=[word_tagged], topn = topn):
                sim_word = i[0].split('_')[0].replace('::', '(-| )')
#                 print(i)
                for topic in new_topics:
                    n = re.search('(\s|^)' + sim_word.lower() + '(\s|$)', topic)
                    if n is not None:
                        result = new_topics[topic]
                        if result not in results:
                            results.append(result)
                        if len(results) > resultsn:
                            return results
    return results

In [58]:
sim_word2vec('boy', 10, 10)

['African American girls',
 'Girls',
 'Chorus girls',
 'Girl Scouts',
 'Infant girls',
 'Teenage girls',
 'Girls -- United States -- 1890-1899',
 'Girls -- 1900-1909',
 'Girls -- 1910-1919',
 'Gibson girls',
 'Girls -- 1870-1879']